### Importing Libraries

In [120]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import Sequential

### Fetching and visualising Data

In [112]:
# Training data
with open('train_image.pkl', 'rb') as f: 
    imgData = pickle.load(f)
with open('train_label.pkl', 'rb') as f:
    imgLabels = pickle.load(f)

In [110]:
# Testing Data
with open('test_image.pkl', 'rb') as f:
    testData = pickle.load(f)

In [113]:
imgData = np.array(imgData)
imgLabels = np.array(imgLabels)
testData = np.array(testData)
print(imgData.shape)
print(imgLabels.shape)
print(testData.shape)

(8000, 784)
(8000,)
(2000, 784)


In [114]:
# As we are later going to convert labels array to array of One-Hot vectors, so we have to change the labels so as to get max value of label = total no. of classes
print(np.unique(imgLabels)) 
for i in range(imgLabels.shape[0]):
    imgLabels[i] = np.ceil(imgLabels[i]/2)
print(np.unique(imgLabels))

[0 2 3 6]
[0 1 2 3]


In [115]:
# As Convolutional layer takes image as input, we have to convert given 1D array to 28 X 28 image
def preprocessData(X,Y):
    X = X.reshape((-1,28,28,1))
    X = X/255.0
    Y = to_categorical(Y) # Converting yTrain to one Hot vector i.e label 0 will be represented as [1 0]
    return X,Y

XTrain,YTrain = preprocess_data(data,labels)
print(XTrain.shape,YTrain.shape)

XTest = testData.reshape((-1,28,28,1))
XTest = XTest/255.0
print(XTest.shape)

(8000, 28, 28, 1) (8000, 3)
(2000, 28, 28, 1)


In [122]:
XTrain, XTest, YTrain, YTest = train_test_split(XTrain, YTrain, test_size=0.2, random_state=35, shuffle=True, stratify=YTrain)

### Training Model

In [127]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
# MaxPool layer is used to reduce the dimensions/no. of features and get the feature with max value from the filter window
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Dropout(0.25)) # It's used to prevent overfitting by randomly removing some neurons in each epoch
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(4,activation='softmax'))
model.summary()

SyntaxError: invalid syntax (<ipython-input-127-c312723c87d0>, line 5)

In [89]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
hist = model.fit(X_train,Y_train,epochs=20,validation_split=0.1,batch_size=128)

Train on 6120 samples, validate on 680 samples
Epoch 1/20
6120/6120 [==============================] - 9s 1ms/step - loss: 0.9612 - acc: 0.5895 - val_loss: 0.8345 - val_acc: 0.6206
Epoch 2/20
6120/6120 [==============================] - 9s 1ms/step - loss: 0.6616 - acc: 0.7059 - val_loss: 0.6664 - val_acc: 0.6926.6742 
Epoch 3/20
6120/6120 [==============================] - 9s 1ms/step - loss: 0.5623 - acc: 0.7691 - val_loss: 0.5879 - val_acc: 0.7603
Epoch 4/20
6120/6120 [==============================] - 9s 1ms/step - loss: 0.4988 - acc: 0.7949 - val_loss: 0.5152 - val_acc: 0.7868
Epoch 5/20
6120/6120 [==============================] - 9s 1ms/step - loss: 0.4545 - acc: 0.8176 - val_loss: 0.5237 - val_acc: 0.7809
Epoch 6/20
6120/6120 [==============================] - 9s 1ms/step - loss: 0.4318 - acc: 0.8317 - val_loss: 0.4870 - val_acc: 0.8074
Epoch 7/20
6120/6120 [==============================] - 9s 1ms/step - loss: 0.4093 - acc: 0.8387 - val_loss: 0.4541 - val_acc: 0.8176
Epoch 8/2

In [90]:
model.evaluate(X_test,Y_test)

1200/1200 [==============================] - 1s 431us/step


[0.3758394642670949, 0.8666666666666667]